In [2]:
import xgboost as xgb
import json
import pandas as pd
import numpy as np

model = xgb.XGBClassifier()
is_model_load = False
weight = {}
is_weight_load = False
df = pd.DataFrame()
is_df_load = False

In [3]:

df = pd.read_csv('DataVis.csv')

In [4]:
df.head(5)

,Unnamed: 0,coords,timestamp_date,year,duration,type_PM2.5,type_การเดินทาง,type_กีดขวาง,type_คนจรจัด,type_คลอง,...,หนองแขม,หลักสี่,ห้วยขวาง,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,"100.66709,13.67891",2021-09-19 14:56:08,2021,274.725706,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
1,1,"100.52649,13.72060",2021-09-26 05:03:52,2021,252.842593,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
2,2,"100.59165,13.82280",2021-12-09 12:29:08,2021,245.784444,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
3,3,"100.64690,13.67083",2021-12-18 14:50:52,2021,1073.560266,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,False
4,4,"100.65440,13.68158",2021-12-22 10:15:33,2021,180.122581,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True


In [5]:
df.columns

Index(['Unnamed: 0', 'coords', 'timestamp_date', 'year', 'duration',
       'type_PM2.5', 'type_การเดินทาง', 'type_กีดขวาง', 'type_คนจรจัด',
       'type_คลอง', 'type_ความปลอดภัย', 'type_ความสะอาด', 'type_จราจร',
       'type_ต้นไม้', 'type_ถนน', 'type_ทางเท้า', 'type_ท่อระบายน้ำ',
       'type_น้ำท่วม', 'type_ป้าย', 'type_ป้ายจราจร', 'type_ร้องเรียน',
       'type_สอบถาม', 'type_สะพาน', 'type_สัตว์จรจัด', 'type_สายไฟ',
       'type_ห้องน้ำ', 'type_เสนอแนะ', 'type_เสียงรบกวน', 'type_แสงสว่าง',
       'คลองสาน', 'คลองสามวา', 'คลองเตย', 'คันนายาว', 'จตุจักร', 'จอมทอง',
       'ดอนเมือง', 'ดินแดง', 'ดุสิต', 'ตลิ่งชัน', 'ทวีวัฒนา', 'ทุ่งครุ',
       'ธนบุรี', 'บางกอกน้อย', 'บางกอกใหญ่', 'บางกะปิ', 'บางขุนเทียน',
       'บางคอแหลม', 'บางซื่อ', 'บางนา', 'บางบอน', 'บางพลัด', 'บางรัก',
       'บางเขน', 'บางแค', 'บึงกุ่ม', 'ปทุมวัน', 'ประเวศ', 'ป้อมปราบศัตรูพ่าย',
       'พญาไท', 'พระนคร', 'พระโขนง', 'ภาษีเจริญ', 'มีนบุรี', 'ยานนาวา',
       'ราชเทวี', 'ราษฎร์บูรณะ', 'ลาดกระบัง', 'ลาดพร้าว', 'ว

In [6]:
district_columns = [
    'คลองสาน', 'คลองสามวา', 'คลองเตย', 'คันนายาว', 'จตุจักร', 'จอมทอง',
    'ดอนเมือง', 'ดินแดง', 'ดุสิต', 'ตลิ่งชัน', 'ทวีวัฒนา', 'ทุ่งครุ',
    'ธนบุรี', 'บางกอกน้อย', 'บางกอกใหญ่', 'บางกะปิ', 'บางขุนเทียน',
    'บางคอแหลม', 'บางซื่อ', 'บางนา', 'บางบอน', 'บางพลัด', 'บางรัก',
    'บางเขน', 'บางแค', 'บึงกุ่ม', 'ปทุมวัน', 'ประเวศ', 'ป้อมปราบศัตรูพ่าย',
    'พญาไท', 'พระนคร', 'พระโขนง', 'ภาษีเจริญ', 'มีนบุรี', 'ยานนาวา',
    'ราชเทวี', 'ราษฎร์บูรณะ', 'ลาดกระบัง', 'ลาดพร้าว', 'วังทองหลาง',
    'วัฒนา', 'สวนหลวง', 'สะพานสูง', 'สัมพันธวงศ์', 'สาทร', 'สายไหม',
    'หนองจอก', 'หนองแขม', 'หลักสี่', 'ห้วยขวาง'
]

weekday_columns = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

# Combine both lists
binary_columns = district_columns + weekday_columns

# Convert True/False to 1/0
df[binary_columns] = df[binary_columns].astype(int)


In [7]:
df.head(5)

,Unnamed: 0,coords,timestamp_date,year,duration,type_PM2.5,type_การเดินทาง,type_กีดขวาง,type_คนจรจัด,type_คลอง,...,หนองแขม,หลักสี่,ห้วยขวาง,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,"100.66709,13.67891",2021-09-19 14:56:08,2021,274.725706,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,"100.52649,13.72060",2021-09-26 05:03:52,2021,252.842593,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,"100.59165,13.82280",2021-12-09 12:29:08,2021,245.784444,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,"100.64690,13.67083",2021-12-18 14:50:52,2021,1073.560266,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,"100.65440,13.68158",2021-12-22 10:15:33,2021,180.122581,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df["distance"] = pd.DataFrame({
    'distance': np.random.randint(1, 10001, size=len(df))
})

In [9]:
display(df)

,Unnamed: 0,coords,timestamp_date,year,duration,type_PM2.5,type_การเดินทาง,type_กีดขวาง,type_คนจรจัด,type_คลอง,...,หลักสี่,ห้วยขวาง,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,distance
0,0,"100.66709,13.67891",2021-09-19 14:56:08,2021,274.725706,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,7976
1,1,"100.52649,13.72060",2021-09-26 05:03:52,2021,252.842593,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4010
2,2,"100.59165,13.82280",2021-12-09 12:29:08,2021,245.784444,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,9191
3,3,"100.64690,13.67083",2021-12-18 14:50:52,2021,1073.560266,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2405
4,4,"100.65440,13.68158",2021-12-22 10:15:33,2021,180.122581,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508051,508051,"100.53687,13.78643",2025-01-15 18:56:28,2025,0.312535,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9250
508052,508052,"100.48659,13.77876",2025-01-15 23:58:03,2025,0.057176,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,334
508053,508053,"100.60907,13.89769",2025-01-15 23:59:13,2025,0.069711,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,372
508054,508054,"100.47857,13.77678",2025-01-16 01:17:56,2025,0.048981,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,4552


In [12]:
def analyze_problem_types(df: pd.DataFrame, problem_types: list[str]):
    df = df.copy()
    df['timestamp_date'] = pd.to_datetime(df['timestamp_date'], errors='coerce')
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
    df.dropna(subset=['timestamp_date', 'duration', 'distance'], inplace=True)

    # --- 1. Quartiles by Day of Week (same as before)
    weekday_cols = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    q_df_by_day = pd.DataFrame(index=weekday_cols, columns=["Q1", "Q2", "Q3", "Q4"])
    mask = df[problem_types].any(axis=1)
    day_df = df[mask]
    for day in weekday_cols:
        d = day_df[day_df[day] == 1]
        if not d.empty:
            q_df_by_day.loc[day] = d['duration'].quantile([0.25, 0.5, 0.75, 1.0]).values

    # --- 2. Average by District (same as before)
    district_cols = [
        'คลองสาน', 'คลองสามวา', 'คลองเตย', 'คันนายาว', 'จตุจักร', 'จอมทอง',
        'ดอนเมือง', 'ดินแดง', 'ดุสิต', 'ตลิ่งชัน', 'ทวีวัฒนา', 'ทุ่งครุ',
        'ธนบุรี', 'บางกอกน้อย', 'บางกอกใหญ่', 'บางกะปิ', 'บางขุนเทียน',
        'บางคอแหลม', 'บางซื่อ', 'บางนา', 'บางบอน', 'บางพลัด', 'บางรัก',
        'บางเขน', 'บางแค', 'บึงกุ่ม', 'ปทุมวัน', 'ประเวศ', 'ป้อมปราบศัตรูพ่าย',
        'พญาไท', 'พระนคร', 'พระโขนง', 'ภาษีเจริญ', 'มีนบุรี', 'ยานนาวา',
        'ราชเทวี', 'ราษฎร์บูรณะ', 'ลาดกระบัง', 'ลาดพร้าว', 'วังทองหลาง',
        'วัฒนา', 'สวนหลวง', 'สะพานสูง', 'สัมพันธวงศ์', 'สาทร', 'สายไหม',
        'หนองจอก', 'หนองแขม', 'หลักสี่', 'ห้วยขวาง'
    ]
    avg_df_by_district = pd.DataFrame(index=district_cols, columns=['Average Duration'])
    for d in district_cols:
        d_df = day_df[day_df[d] == 1]
        avg_df_by_district.loc[d] = d_df['duration'].mean()

    # --- 3. Time Quartiles by Problem Type
    time_grouped = {}
    for pt in problem_types:
        sub = df[(df['timestamp_date'].dt.year >= 2022) & (df['timestamp_date'].dt.year <= 2024)]
        sub = sub[sub[pt] == 1]
        if sub.empty:
            continue
        q_time = (
            sub.set_index('timestamp_date')
            .resample('2W')['duration']
            .agg(Q1=lambda x: x.quantile(0.25),
                 Q2=lambda x: x.quantile(0.5),
                 Q3=lambda x: x.quantile(0.75),
                 Q4=lambda x: x.quantile(1.0))
            .interpolate(method='time')
        )
        time_grouped[pt] = q_time

    # Total (union of all selected types)
    union_df = df[df[problem_types].any(axis=1)]
    union_df = union_df[(union_df['timestamp_date'].dt.year >= 2022) & (union_df['timestamp_date'].dt.year <= 2024)]
    time_grouped["total"] = (
        union_df.set_index('timestamp_date')
        .resample('2W')['duration']
        .agg(Q1=lambda x: x.quantile(0.25),
             Q2=lambda x: x.quantile(0.5),
             Q3=lambda x: x.quantile(0.75),
             Q4=lambda x: x.quantile(1.0))
        .interpolate(method='time')
    )

    # --- 4. Distance Quartiles by Problem Type
    distance_grouped = {}
    for pt in problem_types:
        sub = df[(df['timestamp_date'].dt.year >= 2022) & (df['timestamp_date'].dt.year <= 2024)]
        sub = sub[sub[pt] == 1]
        if sub.empty:
            continue
        max_d = sub['distance'].max()
        bin_width = max_d / 400 if max_d > 0 else 1
        bins = np.arange(0, max_d + bin_width, bin_width)
        sub['distance_bin'] = pd.cut(sub['distance'], bins, right=False)
        q_dist = sub.groupby('distance_bin')['duration'].agg(
            Q1=lambda x: x.quantile(0.25),
            Q2=lambda x: x.quantile(0.5),
            Q3=lambda x: x.quantile(0.75),
            Q4=lambda x: x.quantile(1.0)
        ).interpolate(method='linear')
        distance_grouped[pt] = q_dist

    # Total for distance
    union_df['distance_bin'] = pd.cut(union_df['distance'], np.arange(0, union_df['distance'].max() + bin_width, bin_width), right=False)
    distance_grouped["total"] = union_df.groupby('distance_bin')['duration'].agg(
        Q1=lambda x: x.quantile(0.25),
        Q2=lambda x: x.quantile(0.5),
        Q3=lambda x: x.quantile(0.75),
        Q4=lambda x: x.quantile(1.0)
    ).interpolate(method='linear')

    return q_df_by_day, avg_df_by_district, time_grouped, distance_grouped

In [13]:
types = ['type_น้ำท่วม', 'type_ถนน']
q_day, avg_district, q_time, q_distance = analyze_problem_types(df, types)
display(q_day)
display(avg_district)
display(q_time)
display(q_distance)


C:\Users\asus\AppData\Local\Temp\ipykernel_17188\336668824.py:77: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q_dist = sub.groupby('distance_bin')['duration'].agg(
C:\Users\asus\AppData\Local\Temp\ipykernel_17188\336668824.py:77: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q_dist = sub.groupby('distance_bin')['duration'].agg(
C:\Users\asus\AppData\Local\Temp\ipykernel_17188\336668824.py:87: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and sile

,Q1,Q2,Q3,Q4
Monday,1.812812,8.370706,61.101424,952.238229
Tuesday,1.7361,8.182072,61.244988,933.277222
Wednesday,1.797011,8.562604,63.79702,1021.690035
Thursday,2.117407,9.141562,62.653333,970.986979
Friday,3.006221,9.978721,64.79281,926.83713
Saturday,2.139569,6.918559,53.524925,922.085845
Sunday,1.896696,8.564097,64.922462,938.691505


,Average Duration
คลองสาน,60.319066
คลองสามวา,79.374116
คลองเตย,60.455172
คันนายาว,57.790943
จตุจักร,103.391674
จอมทอง,51.757393
ดอนเมือง,39.855442
ดินแดง,81.270297
ดุสิต,80.964252
ตลิ่งชัน,43.388225


{'type_น้ำท่วม':                         Q1          Q2          Q3          Q4
 timestamp_date                                                
 2022-02-06      281.742593  281.742593  281.742593  281.742593
 2022-02-20      399.675081  436.067402  472.459722  508.852043
 2022-03-06      517.607569  590.392211  663.176852  735.961493
 2022-03-20      433.998372  498.183289  595.954734  722.848412
 2022-04-03      350.389174  405.974367  528.732616  709.735332
 ...                    ...         ...         ...         ...
 2024-11-10        1.767187    3.912674   13.058137   73.196435
 2024-11-24        1.039763    3.097870    7.883536   56.311030
 2024-12-08        1.024687    2.872535    9.941713   44.965671
 2024-12-22        1.725480    3.216007   13.511238   29.826169
 2025-01-05        1.258854    4.496690    9.586887   18.924549
 
 [77 rows x 4 columns],
 'type_ถนน':                         Q1          Q2          Q3           Q4
 timestamp_date                                  

{'type_น้ำท่วม':                          Q1         Q2         Q3          Q4
 distance_bin                                                 
 [0.0, 25.0)        2.587176  14.281435  98.925324  728.866285
 [25.0, 50.0)       2.096134   5.878310  36.097836  780.776458
 [50.0, 75.0)       1.252882   5.492766  29.202188  607.185938
 [75.0, 100.0)      1.763044   4.593762  17.338154  398.200718
 [100.0, 125.0)     1.123131   4.066366  29.893319  454.101204
 ...                     ...        ...        ...         ...
 [9875.0, 9900.0)   1.947049   8.879560  56.951493  385.555116
 [9900.0, 9925.0)   1.407520   6.088594  28.524031  385.086447
 [9925.0, 9950.0)   0.979352   6.040231  55.450185  505.995150
 [9950.0, 9975.0)   1.507951   4.994282  18.153970  419.857662
 [9975.0, 10000.0)  1.751725   7.909624  34.364604  512.934988
 
 [400 rows x 4 columns],
 'type_ถนน':                          Q1        Q2         Q3          Q4
 distance_bin                                                
 [

In [15]:
import plotly.colors as pc

min_val = avg_district['Average Duration'].min()
max_val = avg_district['Average Duration'].max()
avg_district['normalized'] = (avg_district['Average Duration'] - min_val) / (max_val - min_val)

colorscale = pc.get_colorscale('YlOrRd')  

def get_color(value, scale=colorscale):
    return pc.find_intermediate_color(
        scale[0][1],
        scale[-1][1],
        value,
        colortype='rgb'
    )

def rgb_to_hex(rgb_str):
    parts = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    rgb = [round(float(x)) for x in parts]
    return '#%02x%02x%02x' % tuple(rgb)

color_map = {
    district: rgb_to_hex(get_color(norm_val))
    for district, norm_val in avg_district['normalized'].items()
}

print(color_map)

{'คลองสาน': '#e1c3a5', 'คลองสามวา': '#d1a28f', 'คลองเตย': '#e1c2a5', 'คันนายาว': '#e3c7a8', 'จตุจักร': '#bc7874', 'จอมทอง': '#e8d1ae', 'ดอนเมือง': '#f2e6bc', 'ดินแดง': '#cf9e8d', 'ดุสิต': '#cf9f8d', 'ตลิ่งชัน': '#efe0b8', 'ทวีวัฒนา': '#efdfb7', 'ทุ่งครุ': '#ffffcc', 'ธนบุรี': '#f8f2c3', 'บางกอกน้อย': '#e5caaa', 'บางกอกใหญ่': '#e9d2af', 'บางกะปิ': '#ddba9f', 'บางขุนเทียน': '#f5ebbf', 'บางคอแหลม': '#ecdab4', 'บางซื่อ': '#d5ab95', 'บางนา': '#ca9587', 'บางบอน': '#dab49b', 'บางพลัด': '#e1c2a4', 'บางรัก': '#800026', 'บางเขน': '#f1e3ba', 'บางแค': '#efdfb7', 'บึงกุ่ม': '#f3e7bc', 'ปทุมวัน': '#c2857c', 'ประเวศ': '#e9d3af', 'ป้อมปราบศัตรูพ่าย': '#f0e0b8', 'พญาไท': '#cf9f8d', 'พระนคร': '#e6ccab', 'พระโขนง': '#e5cbaa', 'ภาษีเจริญ': '#efdfb7', 'มีนบุรี': '#dbb79d', 'ยานนาวา': '#cf9e8d', 'ราชเทวี': '#cc9a8a', 'ราษฎร์บูรณะ': '#f9f3c4', 'ลาดกระบัง': '#91223c', 'ลาดพร้าว': '#faf4c5', 'วังทองหลาง': '#ebd6b2', 'วัฒนา': '#e9d3af', 'สวนหลวง': '#b76e6e', 'สะพานสูง': '#dbb79d', 'สัมพันธวงศ์': '#fbf7c7', 'สาท

In [18]:
def analyze_duration_in_period(df: pd.DataFrame, start_date: str, end_date: str):
    # Convert and filter by date range
    df = df.copy()
    df['timestamp_date'] = pd.to_datetime(df['timestamp_date'], errors='coerce')
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
    df.dropna(subset=['timestamp_date', 'duration', 'distance'], inplace=True)

    mask = (df['timestamp_date'] >= start_date) & (df['timestamp_date'] <= end_date)
    df = df.loc[mask]

    # --------------------------------
    # 1. Quartiles by day of week
    weekday_cols = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    q_by_day = pd.DataFrame(index=weekday_cols, columns=['Q1', 'Q2', 'Q3', 'Q4'])
    for day in weekday_cols:
        day_df = df[df[day] == 1]
        if not day_df.empty:
            q_by_day.loc[day] = day_df['duration'].quantile([0.25, 0.5, 0.75, 1.0]).values

    # --------------------------------
    # 2. Quartiles by problem type + total
    problem_cols = [col for col in df.columns if col.startswith("type_")]
    q_by_type = pd.DataFrame(index=problem_cols + ['total'], columns=['Q1', 'Q2', 'Q3', 'Q4'])
    for col in problem_cols:
        p_df = df[df[col] == 1]
        if not p_df.empty:
            q_by_type.loc[col] = p_df['duration'].quantile([0.25, 0.5, 0.75, 1.0]).values

    # Add total row
    if not df.empty:
        q_by_type.loc['total'] = df['duration'].quantile([0.25, 0.5, 0.75, 1.0]).values

    # --------------------------------
    # 3. Mean by district
    district_cols = [
        'คลองสาน', 'คลองสามวา', 'คลองเตย', 'คันนายาว', 'จตุจักร', 'จอมทอง',
        'ดอนเมือง', 'ดินแดง', 'ดุสิต', 'ตลิ่งชัน', 'ทวีวัฒนา', 'ทุ่งครุ',
        'ธนบุรี', 'บางกอกน้อย', 'บางกอกใหญ่', 'บางกะปิ', 'บางขุนเทียน',
        'บางคอแหลม', 'บางซื่อ', 'บางนา', 'บางบอน', 'บางพลัด', 'บางรัก',
        'บางเขน', 'บางแค', 'บึงกุ่ม', 'ปทุมวัน', 'ประเวศ', 'ป้อมปราบศัตรูพ่าย',
        'พญาไท', 'พระนคร', 'พระโขนง', 'ภาษีเจริญ', 'มีนบุรี', 'ยานนาวา',
        'ราชเทวี', 'ราษฎร์บูรณะ', 'ลาดกระบัง', 'ลาดพร้าว', 'วังทองหลาง',
        'วัฒนา', 'สวนหลวง', 'สะพานสูง', 'สัมพันธวงศ์', 'สาทร', 'สายไหม',
        'หนองจอก', 'หนองแขม', 'หลักสี่', 'ห้วยขวาง'
    ]
    mean_by_district = pd.DataFrame(index=district_cols, columns=['Mean Duration'])
    for dist in district_cols:
        d_df = df[df[dist] == 1]
        if not d_df.empty:
            mean_by_district.loc[dist] = d_df['duration'].mean()

    # --------------------------------
    # 4. Quartiles over distance (binned)
    max_dist = df['distance'].max()
    bin_width = max_dist / 400
    bins = np.arange(0, max_dist + bin_width, bin_width)
    df['distance_bin'] = pd.cut(df['distance'], bins, right=False)

    q_by_distance = df.groupby('distance_bin')['duration'].agg(
        Q1=lambda x: x.quantile(0.25),
        Q2=lambda x: x.quantile(0.5),
        Q3=lambda x: x.quantile(0.75),
        Q4=lambda x: x.quantile(1.0)
    ).interpolate(method='linear')

    return q_by_day, q_by_type, mean_by_district, q_by_distance

In [25]:
start = "2023-01-01"
end = "2023-12-31"
q_day, q_type, mean_district, q_distance = analyze_duration_in_period(df, start, end)
display(q_day)
display(q_type)
display(mean_district)
display(q_distance)

C:\Users\asus\AppData\Local\Temp\ipykernel_17188\352381317.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q_by_distance = df.groupby('distance_bin')['duration'].agg(


,Q1,Q2,Q3,Q4
Monday,1.780729,8.937859,77.48941,737.99081
Tuesday,1.686846,8.293472,75.660593,730.086123
Wednesday,1.777541,8.878756,76.991085,741.613958
Thursday,1.975891,9.08173,73.780113,742.001007
Friday,2.974167,9.986227,76.292807,726.648299
Saturday,2.152066,6.33463,64.08765,719.467535
Sunday,1.701834,6.894387,60.151317,744.703773


,Q1,Q2,Q3,Q4
type_PM2.5,1.958704,6.379433,26.105058,663.174444
type_การเดินทาง,2.981102,16.271186,133.559392,609.675579
type_กีดขวาง,1.451973,5.976713,59.787928,715.518148
type_คนจรจัด,1.136745,4.017257,16.736047,623.577569
type_คลอง,1.96542,6.043704,46.130515,744.646887
type_ความปลอดภัย,2.774806,15.68798,131.090396,736.659387
type_ความสะอาด,0.975313,2.797188,10.043125,735.443519
type_จราจร,3.021997,30.648698,160.986927,702.977025
type_ต้นไม้,1.713704,4.889699,18.899468,684.754063
type_ถนน,2.136898,10.159213,80.409201,744.703773


,Mean Duration
คลองสาน,75.67998
คลองสามวา,86.721327
คลองเตย,73.076199
คันนายาว,64.652541
จตุจักร,90.882247
จอมทอง,51.584628
ดอนเมือง,47.923071
ดินแดง,84.567599
ดุสิต,78.15175
ตลิ่งชัน,45.282068


,Q1,Q2,Q3,Q4
distance_bin,,,,
"[0.0, 25.0)",2.218736,8.953437,72.153773,705.185463
"[25.0, 50.0)",2.811273,11.815000,110.145289,678.111933
"[50.0, 75.0)",2.011589,7.340185,87.959219,650.898368
"[75.0, 100.0)",1.830301,6.946956,53.187963,704.916840
"[100.0, 125.0)",2.109546,10.071366,101.265561,668.876215
...,...,...,...,...
"[9875.0, 9900.0)",2.062778,9.022442,89.196128,655.869745
"[9900.0, 9925.0)",1.911965,9.972269,98.203683,693.805903
"[9925.0, 9950.0)",1.947002,8.411204,92.722616,567.395799


In [26]:
q_type.index = q_type.index.str.replace('^type_', '', regex=True)
quartiles = q_type.to_dict(orient='index')
print(quartiles)

{'PM2.5': {'Q1': 1.958703703703704, 'Q2': 6.37943287037037, 'Q3': 26.10505787037037, 'Q4': 663.1744444444445}, 'การเดินทาง': {'Q1': 2.981102430555555, 'Q2': 16.271186342592593, 'Q3': 133.55939236111112, 'Q4': 609.6755787037038}, 'กีดขวาง': {'Q1': 1.4519733796296297, 'Q2': 5.976712962962963, 'Q3': 59.78792824074074, 'Q4': 715.5181481481482}, 'คนจรจัด': {'Q1': 1.1367447916666666, 'Q2': 4.017256944444444, 'Q3': 16.7360474537037, 'Q4': 623.5775694444444}, 'คลอง': {'Q1': 1.9654195601851854, 'Q2': 6.043703703703703, 'Q3': 46.130515046296296, 'Q4': 744.6468865740741}, 'ความปลอดภัย': {'Q1': 2.774806134259259, 'Q2': 15.687980324074076, 'Q3': 131.09039641203705, 'Q4': 736.6593865740741}, 'ความสะอาด': {'Q1': 0.9753125, 'Q2': 2.7971875, 'Q3': 10.043125, 'Q4': 735.4435185185185}, 'จราจร': {'Q1': 3.021996527777777, 'Q2': 30.64869791666667, 'Q3': 160.98692708333334, 'Q4': 702.977025462963}, 'ต้นไม้': {'Q1': 1.7137037037037035, 'Q2': 4.889699074074074, 'Q3': 18.899467592592597, 'Q4': 684.7540625}, 'ถน

In [53]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def analyze_by_districts(df, districts):
    # Filter rows that match any of the given districts
    district_mask = df[districts].sum(axis=1) > 0
    filtered_df = df[district_mask].copy()

    # Get list of problem types and weekdays
    problem_types = [col for col in df.columns if col.startswith("type_")]
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    ### 1. Quartiles of duration per problem type ###
    q_problem = filtered_df.groupby(
        lambda x: next((col for col in problem_types if filtered_df.loc[x, col] == 1), None)
    )['duration'].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.25: 'Q1', 0.5: 'Q2', 0.75: 'Q3'})

    # Handle multiple problem types per row
    rows = []
    for t in problem_types:
        subset = filtered_df[filtered_df[t] == 1]
        if not subset.empty:
            q = subset['duration'].quantile([0.25, 0.5, 0.75])
            rows.append((t, *q.values))
    df_q1 = pd.DataFrame(rows, columns=['ProblemType', 'Q1', 'Q2', 'Q3'])
    total_q = filtered_df['duration'].quantile([0.25, 0.5, 0.75])
    df_q1.loc[len(df_q1)] = ['Total', *total_q.values]

    ### 2. Quartiles of duration per weekday ###
    rows = []
    for day in weekdays:
        subset = filtered_df[filtered_df[day] == 1]
        if not subset.empty:
            q = subset['duration'].quantile([0.25, 0.5, 0.75])
            rows.append((day, *q.values))
    df_q2 = pd.DataFrame(rows, columns=['Weekday', 'Q1', 'Q2', 'Q3'])

    ### 3. Quartiles of duration over time (uniformly sampled) ###
    time_min, time_max = filtered_df['timestamp_date'].min(), filtered_df['timestamp_date'].max()
    time_range = pd.date_range(start=time_min, end=time_max, periods=401)
    df_time = filtered_df[['timestamp_date', 'duration']].copy()
    df_time = df_time.sort_values('timestamp_date')
    filtered_df['timestamp_date'] = pd.to_datetime(filtered_df['timestamp_date'])

    # Interpolation function over time
    filtered_df['timestamp_date'] = pd.to_datetime(filtered_df['timestamp_date'])

    # Now get the min and max (after conversion!)
    time_min = pd.to_datetime(df_time['timestamp_date'].min())
    time_max = pd.to_datetime(df_time['timestamp_date'].max())
    time_points = np.linspace(0, (time_max - time_min).total_seconds(), 401)
    df_q3 = pd.DataFrame({'timestamp': time_range})
    for q in [0.25, 0.5, 0.75]:
        grouped = df_time.groupby('timestamp_date')['duration'].quantile(q)
        grouped = grouped.reset_index()
        grouped['timestamp_date'] = pd.to_datetime(grouped['timestamp_date'])  # Ensure datetime
        grouped['timestamp_num'] = (grouped['timestamp_date'] - pd.Timestamp(time_min)).dt.total_seconds()
        f = interp1d(grouped['timestamp_num'], grouped['duration'], kind='linear', fill_value='extrapolate')
        df_q3[f'Q{int(q*4)}'] = f(time_points)

    ### 4. Quartiles of duration over distance (uniformly sampled) ###
    dist_min, dist_max = filtered_df['distance'].min(), filtered_df['distance'].max()
    dist_points = np.linspace(dist_min, dist_max, 401)
    df_q4 = pd.DataFrame({'distance': dist_points})
    for q in [0.25, 0.5, 0.75]:
        grouped = filtered_df.groupby('distance')['duration'].quantile(q).reset_index()
        f = interp1d(grouped['distance'], grouped['duration'], kind='linear', fill_value='extrapolate')
        df_q4[f'Q{int(q*4)}'] = f(dist_points)

    return df_q1, df_q2, df_q3, df_q4

In [ ]:
districts = ['คลองสาน', 'คลองเตย', 'จตุจักร']
q_type, q_day, q_time, q_distance = analyze_by_districts(df, districts)
display(df_q1)
display(df_q2)
display(df_q3)
display(df_q4)

,ProblemType,Q1,Q2,Q3
0,type_PM2.5,2.991748,9.037963,23.853886
1,type_การเดินทาง,2.954994,13.429161,78.753223
2,type_กีดขวาง,1.032894,5.017812,51.056759
3,type_คนจรจัด,0.846505,2.587130,9.982616
4,type_คลอง,3.125000,13.712535,108.186366
5,type_ความปลอดภัย,2.313397,13.331852,112.493605
6,type_ความสะอาด,1.208426,5.306852,26.885394
7,type_จราจร,1.728987,13.187662,127.914502
8,type_ต้นไม้,2.175194,7.102031,36.925162
9,type_ถนน,2.674502,13.115833,90.620584


,Weekday,Q1,Q2,Q3
0,Monday,1.820162,10.471366,94.661134
1,Tuesday,1.762066,9.906134,98.679745
2,Wednesday,1.776528,9.331528,84.636319
3,Thursday,2.856380,11.909306,93.538255
4,Friday,2.824792,11.102940,88.246568
5,Saturday,2.588993,11.244363,107.535949
6,Sunday,1.883623,10.340069,107.379595


,timestamp,Q1,Q2,Q3
0,2022-02-15 02:47:41.000000,749.379525,749.379525,749.379525
1,2022-02-17 18:41:58.942500,749.449936,749.449936,749.449936
2,2022-02-20 10:36:16.885000,749.520347,749.520347,749.520347
3,2022-02-23 02:30:34.827500,749.590757,749.590757,749.590757
4,2022-02-25 18:24:52.770000,749.661168,749.661168,749.661168
...,...,...,...,...
396,2025-01-04 13:10:06.230000,3.687467,3.687467,3.687467
397,2025-01-07 05:04:24.172500,1.516428,1.516428,1.516428
398,2025-01-09 20:58:42.115000,2.607249,2.607249,2.607249
399,2025-01-12 12:53:00.057500,1.358034,1.358034,1.358034


,distance,Q1,Q2,Q3
0,1.0000,1.937937,6.459132,21.338686
1,25.9975,10.870725,59.701125,257.181726
2,50.9950,1.617933,8.058286,15.200609
3,75.9925,2.284584,8.836520,12.555371
4,100.9900,1.616650,2.173026,58.315332
...,...,...,...,...
396,9900.0100,7.851571,15.009299,285.750351
397,9925.0075,0.652117,6.079322,39.298658
398,9950.0050,4.523091,5.893060,12.807493
399,9975.0025,1.839957,103.720631,207.099581
